In [ ]:
from numpy.random import seed
seed(1)

import tensorflow
tensorflow.random.set_seed(3)

from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from tensorflow.keras.activations import softmax

In [ ]:
import import_ipynb

from preprocessing import *

In [ ]:
def get_seq_responses(raw_data, seq_length=20, emb_dim=32, max_responses=7):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['responses'], seq_length=seq_length)
    
    feature_length = len(fewer_plot_names) 
    seq_responses = Sequential()
    seq_responses.add(InputLayer(input_shape=(max_responses, emb_dim,)))
    seq_responses.add(GRU(16, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    seq_responses.add(Dense(8, activation='relu'))
    seq_responses.add(Dropout(0.2))
    
    #pretest_input = Input(shape=1)
    
    #merged = Concatenate()([logs_responses.output, pretest_input])
    
    dense2 = Dense(8)(seq_responses.output)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(dense2)
    
    combined_model = Model(seq_responses.input, outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_only_sequences(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['seq_goals'], seq_length=seq_length)
    
    feature_length = len(fewer_plot_names) 
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(feature_length, feature_length,)))
    logs_responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    #logs_responses.add(Dense(16, activation='relu'))
    logs_responses.add(Dropout(0.5))
    
    #pretest_input = Input(shape=1)
    
    #merged = Concatenate()([logs_responses.output, pretest_input])
    
    #dense2 = Dense(8)(logs_responses.output)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(logs_responses.output)
    
    combined_model = Model(logs_responses.input, outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_seqgoals_responses(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['seq_goals', 'responses'], seq_length=seq_length)
    
    feature_length = len(fewer_plot_names) 
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(feature_length, feature_length,)))
    logs_responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    #logs_responses.add(Dense(16, activation='relu'))
    logs_responses.add(Dropout(0.2))
    
    response_input = Input(shape=emb_dim)
    dense_response = Dense(16)(response_input)
    dense_response = Dropout(0.2)(dense_response)
    
    merged = Concatenate()([logs_responses.output, dense_response])
    
    #dense2 = Dense(16)(merged)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(merged)
    
    combined_model = Model([logs_responses.input, response_input], outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_all_model(raw_data, seq_length=20, emb_dim=32, max_responses=7):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['seq_goals', 'responses', 'seq_locnevents'], 
                                        seq_length=seq_length)
    
    feature_length = len(fewer_plot_names) 
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(feature_length, feature_length,)))
    logs_responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    logs_responses.add(Dropout(0.2))
    
    responses = Sequential()
    responses.add(InputLayer(input_shape=(max_responses, emb_dim,)))
    responses.add(GRU(16, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    responses.add(Dropout(0.2))
    
    seq_locns = Sequential()
    seq_locns.add(InputLayer(input_shape=(seq_length, len(locations)+len(events_without_movement),)))
    seq_locns.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    
    merged1 = Concatenate()([responses.output, seq_locns.output])
    
    dense_response = Dense(16)(merged1)
    dense_response = Dropout(0.5)(dense_response)
    
    merged = Concatenate()([dense_response, logs_responses.output])
    
    dense2 = Dense(16)(merged)
    #dense3 = Dense(8)(dense2)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(dense2)
    
    combined_model = Model([logs_responses.input, responses.input, seq_locns.input], outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_sequences_locn(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['seq_goals', 'current_locns'], 
                                        seq_length=seq_length)
    
    feature_length = len(fewer_plot_names) 
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(feature_length, feature_length,)))
    logs_responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    #logs_responses.add(Dense(16, activation='relu'))
    logs_responses.add(Dropout(0.2))
    
    locn_input = Input(shape=len(locations))
    
    merged = Concatenate()([logs_responses.output, locn_input])
    
    #dense2 = Dense(16)(merged)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(merged)
    
    combined_model = Model([logs_responses.input, locn_input], outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_sequences_seqlocn(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['seq_goals', 'seq_locns'], seq_length=seq_length)
    
    feature_length = len(fewer_plot_names) 
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(feature_length, feature_length,)))
    logs_responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    #logs_responses.add(Dense(16, activation='relu'))
    logs_responses.add(Dropout(0.2))
    
    seq_locns = Sequential()
    seq_locns.add(InputLayer(input_shape=(seq_length, len(locations),)))
    seq_locns.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    #logs_responses.add(Dense(16, activation='relu'))
    seq_locns.add(Dropout(0.2))
    
    merged = Concatenate()([logs_responses.output, seq_locns.output])
    
    #dense2 = Dense(16)(merged)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(merged)
    
    combined_model = Model([logs_responses.input, seq_locns.input], outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_sequences_seqlocnevent(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['seq_goals', 'seq_locnevents'],
                                        seq_length=seq_length)
    
    feature_length = len(fewer_plot_names) 
    logs_responses = Sequential()
    logs_responses.add(InputLayer(input_shape=(feature_length, feature_length,)))
    logs_responses.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    #logs_responses.add(Dense(16, activation='relu'))
    logs_responses.add(Dropout(0.2))
    
    seq_locns = Sequential()
    seq_locns.add(InputLayer(input_shape=(seq_length, len(locations)+len(events_without_movement),)))
    seq_locns.add(GRU(32, return_sequences=False, dropout=0.1,
                    recurrent_dropout=0.1, kernel_regularizer=l2(0.01), 
                    recurrent_regularizer=l2(0.01), 
                    bias_regularizer=l2(0.01)))
    seq_locns.add(Dense(16, activation='relu'))
    seq_locns.add(Dropout(0.2))
    
    merged = Concatenate()([logs_responses.output, seq_locns.output])
    
    #dense2 = Dense(16)(merged)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(merged)
    
    combined_model = Model([logs_responses.input, seq_locns.input], outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_dist_model(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['dist'], seq_length=seq_length)
    
    dist_input = Input(shape=len(fewer_plot_names))
    
    #merged = Concatenate()([logs_responses.output, dist_input])
    
    dense2 = Dense(16)(dist_input)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(dist_input)
    
    combined_model = Model(dist_input, outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_locn_model(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['current_locns'], seq_length=seq_length)
    
    locn_input = Input(shape=len(locations))
    
    #merged = Concatenate()([logs_responses.output, dist_input])
    
    #dense2 = Dense(16)(dist_input)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(locn_input)
    
    combined_model = Model(locn_input, outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids

In [ ]:
def get_responses_model(raw_data, seq_length=20, emb_dim=32):
    # Preprocess data to suit model 
    data, labels, ids = preprocess_data(raw_data, ['last_responses'], seq_length=seq_length)
    
    response_input = Input(shape=emb_dim)
    
    #merged = Concatenate()([logs_responses.output, dist_input])
    
    #dense2 = Dense(16)(dist_input)
    
    output_goals = Dense(len(fewer_plot_names), activation='softmax', name='goal_label')(response_input)
    
    combined_model = Model(response_input, outputs=output_goals) 
    
    optimizer = Adam(lr=0.01)
    
    #loss_weights = {} [ADD IF REQUIRED]
    
    combined_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) #, loss_weights=loss_weights)
    
    return combined_model, data, labels, ids